In [38]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.get_details()

exp = Experiment(workspace=ws, name="optim-hdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-143750
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-143750


In [39]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name = "hdrive-clus"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [40]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.001, 0.01, .1, .3, .5, 1.0),
    '--max_iter': choice(50, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
              entry_script='train.py',
              compute_target=compute_target,
              vm_size='Standard_D2_V2')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs=4)

In [41]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hd_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [48]:
import joblib
# Get your best run and save the model from that run.

best_hd_run = hd_run.get_best_run_by_primary_metric()

print(f"Best Run Metrics : {best_hd_run.get_metrics()}")

best_hd_run.download_file(
    best_hd_run.get_file_names()[-1],
    output_file_path='./outputs/'
)

best_hd_model = best_hd_run.register_model(
        model_name='best_hyperdrive_model',
        model_path='./outputs/best_hyperdrive_model.joblib'
)

Best Run Metrics : {'Regularization Strength:': 0.5, 'Max iterations:': 50, 'Accuracy': 0.9093226511289147}


In [49]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files(path=data_path)

In [50]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(data)

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [53]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=data,
    label_column_name='y',
    n_cross_validations=5,
    enable_onnx_compatible_models=True)

In [54]:
# Submit your automl run

exp_automl = Experiment(ws, 'automl')
automl_run = exp_automl.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on hdrive-clus with default configuration
Running on remote compute: hdrive-clus


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_db8f415c-4041-44dc-abcb-ff25a9a2513b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [ ]:
# Retrieve and save your best automl model.

best_run, best_model = automl_run.get_output()

automl_model = best_run.register_model(
    model_name='best_automl_model.pkl',
    model_output='./outputs'
    )